In [1]:
from energyflow.archs.moment import EFN_moment, PFN_moment
from energyflow.archs.moment_layers import Moment
from sklearn.metrics import roc_auc_score
from utils.data_utils import load_data
try:
    from config import base_dir
except:
    raise ValueError("Please specify a base analysis directory, base_dir, in config.py!")

import numpy as np
import tensorflow as tf
import sys
import os


print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("GPUs Available: ", tf.config.list_physical_devices('GPU'))

2023-09-10 15:16:09.341194: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-10 15:16:09.342547: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-10 15:16:09.368914: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-10 15:16:09.369592: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-10 15:16:09.825826: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

Num GPUs Available:  0
GPUs Available:  []


In [ ]:
# Turn off TEST_MODE to run on full dataset
TEST_MODE = True

# Parameters 
train = 1000
val = 5000
test = 5000
is_EFN = True
categorical = False
dataset = "qg"

# Load data
(X_train, X_val, X_test,), (Y_train, Y_val, Y_test,) = load_data(dataset, train, val, test, 
                                                                 EFN_format=is_EFN, 
                                                                 categorical=categorical)
print("Data loaded!")

In [2]:
new_model = tf.keras.models.load_model('Data/test/order_1/Models/O1_L1_2Phi25_3F25_0.keras', custom_objects={'Moment': Moment})

# Show the model architecture
new_model.summary()

# 


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, 2)]    0           []                               
                                                                                                  
 tdist0_0 (TimeDistributed)     (None, None, 25)     75          ['input_1[0][0]']                
                                                                                                  
 activation (Activation)        (None, None, 25)     0           ['tdist0_0[0][0]']               
                                                                                                  
 tdist0_1 (TimeDistributed)     (None, None, 25)     650         ['activation[0][0]']             
                                                                                              

In [3]:
if dataset == "qg":
    X, Y = qg_jets.load(train+val+test, cache_dir="/n/holyscratch01/iaifi_lab/rikab/.energyflow")
    X = X[:,:,:3].astype(np.float32)
    for x in X:
        mask = x[:,0] > 0
        yphi_avg = average(x[mask,1:3], weights=x[mask,0], axis=0)
        x[mask,1:3] -= yphi_avg
        x[mask,0] /= x[:,0].sum()


    Y = to_categorical(Y, num_classes=2)

    (z_train, z_val, z_test,
    p_train, p_val, p_test,
    Y_train, Y_val, Y_test) = data_split(X[:,:,0], X[:,:,1:], Y, val=val, test=test)

    X_train = [z_train, p_train]
    X_val = [z_val, p_val]
    X_test = [z_test, p_test]

elif dataset == "top":
    X_train, Y_train = toploader.load(cache_dir=topdir, dataset="train", num_data = train)
    X_test, Y_test = toploader.load(cache_dir=topdir, dataset="test", num_data=test)
    X_val, Y_val = toploader.load(cache_dir=topdir, dataset="val", num_data=test)

    def format(X):

        for x in X:
            mask = x[:,0] > 0
            yphi_avg = average(x[mask,1:3], weights=x[mask,0], axis=0)
            x[mask,1:3] -= yphi_avg
            x[mask,0] /= x[:,0].sum()

        return [X[:,:,0], X[:,:,1:3]]
    
    X_train = format(X_train)
    X_test = format(X_test)
    X_val = format(X_val)

    Y_train = to_categorical(Y_train, num_classes=2)
    Y_test = to_categorical(Y_test, num_classes=2)
    Y_val = to_categorical(Y_val, num_classes=2)


else:
    raise ValueError("dataset must be either `top` or `qg`!")


print("Data loaded!")

NameError: name 'dataset' is not defined

In [ ]:
## training

order = 1
L, F, Phi = 1, 100, 100
print(L, F, Phi, order)

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=8)

model_name = f"O{order}_L{L}_2Phi{Phi}_3F{F}"
container = ModelsContainer(**{'Phi_mapping_dim' : [input_dim,L],
                            'output_dim' : 2, 'output_act' : 'softmax',
                            'Phi_sizes' : [Phi, Phi], 'Phi_acts' : 'ReLU', "Phi_l1_regs" :  1e-7,
                            'F_sizes' : [F,F,F], 'F_acts': 'ReLU', "F_l1_regs" :  1e-7,
                            'order' : order , 'architecture_type':'moment',
                            'loss': 'binary_crossentropy','metrics': 'acc','metrics': ['acc', tf.keras.metrics.AUC()]})
print()
print(i, order, container.num_params)
print()
mean, std = container.train_models(num_models = num_models_to_train,
                            X_train = X_train, Y_train = Y_train,
                            epochs = 5, batch_size = batch_size,
                            path = os.path.join(model_dir , model_name),
                            validation_data = (X_val, Y_val),
                            callbacks=[callback,], verbose=verbose,
                            metric_function = roc_auc_score)
# container.test_meanstd(X_test = X_test, Y_test = Y_test, metric_function = roc_auc_score)
print(mean, std)



1 100 100 1

7 1 31102

Epoch 1/5
1/1 - 3s - loss: 0.7523 - acc: 0.5000 - auc_2: 0.5000 - val_loss: 0.7009 - val_acc: 0.5034 - val_auc_2: 0.5000 - 3s/epoch - 3s/step
Epoch 2/5
1/1 - 1s - loss: 0.7018 - acc: 0.5000 - auc_2: 0.5000 - val_loss: 0.6952 - val_acc: 0.4966 - val_auc_2: 0.5000 - 714ms/epoch - 714ms/step
Epoch 3/5
1/1 - 1s - loss: 0.6949 - acc: 0.5000 - auc_2: 0.5000 - val_loss: 0.7075 - val_acc: 0.4966 - val_auc_2: 0.5000 - 713ms/epoch - 713ms/step
Epoch 4/5
1/1 - 1s - loss: 0.7064 - acc: 0.5000 - auc_2: 0.5000 - val_loss: 0.7133 - val_acc: 0.4966 - val_auc_2: 0.5000 - 1s/epoch - 1s/step
Epoch 5/5
1/1 - 1s - loss: 0.7120 - acc: 0.5000 - auc_2: 0.5000 - val_loss: 0.7097 - val_acc: 0.4966 - val_auc_2: 0.5000 - 1s/epoch - 1s/step
157/157 [==============================] - 1s 7ms/step
Epoch 1/5
1/1 - 2s - loss: 0.8181 - acc: 0.5000 - auc_2: 0.5000 - val_loss: 0.7566 - val_acc: 0.4966 - val_auc_2: 0.5000 - 2s/epoch - 2s/step
Epoch 2/5
1/1 - 1s - loss: 0.7542 - acc: 0.5000 - auc_2: 